In [144]:
!pip install -r https://raw.githubusercontent.com/mitramir55/PassivePy/main/PassivePyCode/PassivePySrc/requirements_lg.txt
!pip install PassivePy==0.2.20

     |████████████████████████████████| 6.5 MB 566 kB/s eta 0:00:01     |█████████████████▍              | 3.5 MB 82 kB/s eta 0:00:37
     |████████████████████████████████| 772 kB 6.3 MB/s eta 0:00:01
     |████████████████████████████████| 587.7 MB 23 kB/s  eta 0:00:012    |█████▉                          | 107.1 MB 632 kB/s eta 0:12:41████                 | 275.5 MB 9.8 MB/s eta 0:00:32    |███████████████▊                | 287.8 MB 634 kB/s eta 0:07:53    |███████████████▊                | 289.8 MB 618 kB/s eta 0:08:02    |███████████████▉                | 291.4 MB 567 kB/s eta 0:08:43  | 312.9 MB 604 kB/s eta 0:07:35��███████▏              | 315.6 MB 589 kB/s eta 0:07:42��███████▎              | 316.8 MB 589 kB/s eta 0:07:40��█▎            | 353.6 MB 19.4 MB/s eta 0:00:13   | 355.7 MB 626 kB/s eta 0:06:11   | 357.1 MB 547 kB/s eta 0:07:02�███████▋            | 359.2 MB 547 kB/s eta 0:06:58�████████▏           | 369.5 MB 1.3 MB/s eta 0:02:42�████████▋           | 378.3 MB 4.5 MB/s 

In [145]:
from os import listdir
from os.path import isfile, join
import json
import textstat
from unidecode import unidecode
import pandas as pd
import nltk
from matplotlib import pyplot as plt

from PassivePySrc import PassivePy

passivepy = PassivePy.PassivePyAnalyzer(spacy_model = "en_core_web_lg")

/home/monty/.local/lib/python3.8/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_lg' (3.4.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
onlyfiles = [f for f in listdir("/home/monty/RyersonGraduateStudies/CP8207/Final Project/fire-and-blood-story-generation/data/processed-data/context-data") if isfile(join("/home/monty/RyersonGraduateStudies/CP8207/Final Project/fire-and-blood-story-generation/data/processed-data/context-data", f))]

In [70]:
characters = pd.read_csv("/home/monty/RyersonGraduateStudies/CP8207/Final Project/fire-and-blood-story-generation/data/processed-data/fabNames.csv")

In [121]:
named_characters = list(characters.loc[characters["NUN"]=="Named"]["FullName"].apply(lambda x:x.split("/")[0]).values)
unnamed_characters = list(characters.loc[characters["NUN"]=="US"]["FullName"].apply(lambda x:x.split("/")[0]).values)

In [142]:
for x in unnamed_characters:
    if "bard" in x:
        print(x)

In [137]:
def character_accuracy(path):
    entity_correctness = {"tp":0.0, "fp":0.0, "fn":0.0}
    unnamed_tp = 0
    character_list = []
    for json_file in onlyfiles:
        with open(f"{path}/{json_file}", 'r') as f:
            data = json.load(f)
            data = data[json_file.split(".")[0]]
            for paragraph in data:
                decoded_text = unidecode(paragraph["paragraph"])
                for character in paragraph["characters"]:
                    if (character in named_characters) and character in decoded_text:
                        entity_correctness["tp"] += 1
                    if character in unnamed_characters:
                        unnamed_tp += 1
                    character_list.append(character)
                for true_character in named_characters:  
                    if true_character in decoded_text and true_character not in paragraph["characters"]:
                        entity_correctness["fn"] += 1
    non_named_characters = [character for character in list(set(character_list)) if character not in named_characters]
    print(len(non_named_characters))

        
    print(entity_correctness['tp'])
    print(entity_correctness['fp'])
    
    print(f"Named Character Recall: {entity_correctness['tp']/len(named_characters)}")
    print(f"Unnamed Character Recall: {unnamed_tp/len(unnamed_characters)}")
    print(f"Named Character False Negatives: {entity_correctness['fn']}")

In [138]:
character_accuracy("/home/monty/RyersonGraduateStudies/CP8207/Final Project/fire-and-blood-story-generation/data/processed-data/context-data")

2074
250.0
0.0
Named Character Recall: 0.9398496240601504
Unnamed Character Recall: 0.0
Named Character False Negatives: 15.0


In [104]:
def POS_accuracy(path):
    pos_correctness = {"verb":0.0, "adj":0.0, "location"}
    total_verbs = 0
    total_adjs = 0

    adj_lens = []
    verb_lens = []
    for json_file in onlyfiles:
        with open(f"{path}/{json_file}", 'r') as f:
            data = json.load(f)
            data = data[json_file.split(".")[0]]
            for paragraph in data:
                decoded_text = unidecode(paragraph["paragraph"])
                tokenized_text = nltk.word_tokenize(decoded_text)
                pos_tagged = nltk.pos_tag(tokenized_text)
                verbs = filter(lambda x:(x[1]=='VB' or x[1]=='VBD' or x[1]=='VBN' or x[1]=='VBP' or x[1]=='VBZ'),pos_tagged)
                verbs = [verb[0] for verb in tuple(verbs)]
                adjectives = filter(lambda x:(x[1]=='JJ' or x[1]=='JJR' or x[1]=='JJS'),pos_tagged)
                adjectives = [adjec[0] for adjec in tuple(adjectives)]
                total_verbs += len(set(verbs))
                total_adjs += len(set(adjectives))
                for verb in paragraph["verbs"]:
                    verb_lens.append(len(verb.split()))
                    if verb in verbs:
                        pos_correctness["verb"] += 1
                for adj in paragraph["adjectives"]:
                    adj_lens.append(len(adj.split()))
                    if adj in adjectives:
                        pos_correctness["adj"] += 1
        
    print(f"Verb Accuracy: {pos_correctness['verb']/total_verbs}")
    print(f"Adjective Accuracy: {pos_correctness['adj']/total_adjs}")  
    print(f"Verb Length Mean: {sum(verb_lens)/len(verb_lens)}")  
    print(f"Verb Length Max: {max(verb_lens)}") 
    print(f"Verb Length Min: {min(verb_lens)}")     
    print(f"Adj Length Mean: {sum(adj_lens)/len(adj_lens)}")  
    print(f"Adj Length Max: {max(adj_lens)}") 
    print(f"Adj Length Min: {min(adj_lens)}")     

In [105]:
POS_accuracy("/home/monty/RyersonGraduateStudies/CP8207/Final Project/fire-and-blood-story-generation/data/processed-data/context-data")

Verb Accuracy: 0.2205746061167748
Adjective Accuracy: 0.2285982264665757
Verb Length Mean: 1.6624663332050789
Verb Length Max: 47
Verb Length Min: 1
Adj Length Mean: 1.4362184721466158
Adj Length Max: 66
Adj Length Min: 1


In [ ]:
def passivity(path):
    passive_counts = []
    for json_file in onlyfiles:
        with open(f"{path}/{json_file}", 'r') as f:
            data = json.load(f)
            data = data[json_file.split(".")[0]]
            for paragraph in data:
                decoded_text = unidecode(paragraph["paragraph"])
